In [1]:
import torch
import torchvision.transforms as T
import torchmultimodal.models.omnivore as omnivore

from PIL import Image
import collections
import json
import matplotlib.pyplot as plt
import numpy as np


In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def custom_load_state_dict(model, pretrained_state_dict):
    # Convert the pretrained_state_dict so it have the same keys as the model
    # then load the value of the weight into the model
    pretrained_keys = list(pretrained_state_dict.keys())
    model_keys = list(model.state_dict().keys())
    key_mapping = {pretrained_keys[i]: model_keys[i] for i in range(len(model_keys))}
    updated_pretrained_state_dict = collections.OrderedDict({key_mapping[key]: val for key, val in pretrained_state_dict.items()})
    model.load_state_dict(updated_pretrained_state_dict)

In [3]:
# Load model from torch_hub
mhub = torch.hub.load("facebookresearch/omnivore:main", model="omnivore_swinT")
mhub.eval()
print(count_parameters(mhub))

Using cache found in /Users/yosuamichael/.cache/torch/hub/facebookresearch_omnivore_main
/Users/yosuamichael/.pyenv/versions/3.9.10/envs/multimodal-3.9/lib/python3.9/site-packages/torch/functional.py:476: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:2353.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


28945041


In [5]:
m = omnivore.omnivore_swin_t()

# Check that it have same number of parameter
print(count_parameters(m))

28945041


In [6]:
custom_load_state_dict(m, mhub.state_dict())
m = m.eval()


/Users/yosuamichael/.pyenv/versions/3.9.10/envs/multimodal-3.9/lib/python3.9/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


# Inference test

In [7]:
# Download imagenet class and image
# Uncomment to download
!wget https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json -O imagenet_class_index.json
with open("imagenet_class_index.json", "r") as f:
    imagenet_classnames = json.load(f)

# Create an id to label name mapping
imagenet_id_to_classname = {}
for k, v in imagenet_classnames.items():
    imagenet_id_to_classname[k] = v[1] 

# Download the example image file
# Uncomment to download
!wget -O library.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/c/c5/13-11-02-olb-by-RalfR-03.jpg/800px-13-11-02-olb-by-RalfR-03.jpg

image_path = "library.jpg"
image_pil = Image.open(image_path).convert("RGB")
plt.figure(figsize=(6, 6))
plt.imshow(image_pil)

--2022-05-13 11:50:44--  https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 64:ff9b::34d9:aee8, 52.216.140.118
Connecting to s3.amazonaws.com (s3.amazonaws.com)|64:ff9b::34d9:aee8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35363 (35K) [application/octet-stream]
Saving to: ‘imagenet_class_index.json’

imagenet_class_inde 100%[===================>]  34.53K  --.-KB/s    in 0.08s   

2022-05-13 11:50:44 (410 KB/s) - ‘imagenet_class_index.json’ saved [35363/35363]



FileNotFoundError: [Errno 2] No such file or directory: 'library.jpg'

In [ ]:
image_transform = T.Compose(
    [
        T.Resize(224),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
image = image_transform(image_pil)  # C H W

# Adding batch and time (D) dimension
image = image.unsqueeze(0).unsqueeze(2)  # B C D H W

In [ ]:
def infer(model):
    with torch.no_grad():
        prediction = model(image, input_type="image")
        pred_classes = prediction.topk(k=5).indices

    pred_class_names = [imagenet_id_to_classname[str(i.item())] for i in pred_classes[0]]
    print("Top 5 predicted labels: %s" % ", ".join(pred_class_names))

In [ ]:
# Test both model to infer the same image and make sure the output classes are the same
infer(m)
infer(mhub)

# Make sure the output of the trunk / encoder are the same

In [ ]:
m_feature = m.encoder(image)
mhub_feature = mhub.trunk(image)

In [ ]:
# See the first 10 features are the same
m_feature.flatten()[:10], mhub_feature[0].flatten()[:10]

In [ ]:
# Make sure all the features are the same
np.all(np.array(m_feature == mhub_feature[0]))